# Import librairies

In [1]:
import pandas as pd
import numpy as np
import evaluate
from transformers import AutoTokenizer, AutoModel, pipeline
from transformers import PLBartForConditionalGeneration, PLBartTokenizer
from transformers import TrainingArguments,Trainer
from datasets import load_dataset

In [3]:
DATA_PATH_1 = "../data/data_pairs_1.csv"
DATA_PATH_2 = "../data/data_pairs_2.csv"

# Import Data

In [4]:
df_1 = pd.read_csv(DATA_PATH_1).drop(['Unnamed: 0','title'],axis=1)
df_1

,markdown,code
0,House Prices: Advanced Regression Techniques P...,import numpy as np
1,"data processing, CSV file I O e.g. pd.read csv",import pandas as pd
2,Input data files are available in the .. input...,from datetime import datetime
3,for some statistics,from scipy . stats import skew from scipy ....
4,Any results you write to the current directory...,train = pd.read_csv('../input/house-prices-adv...
...,...,...
7567,"If the word exists in the vocabulary, this com...",test_features = tfidf.transform(X_test).toarra...
7568,reduce the matrix dimensionality in order to ...,"""""""from sklearn import feature_selection \r\ny..."
7569,Model training Model : Multinomial NB,import pandas as pd\r\nfrom sklearn.naive_baye...
7570,Visualizing scikit model performance,training_accuracy = accuracy_score(train_label...


# Import the pretrained model "PLBART" and its tokenizer

In [5]:
plbarttokenizer = PLBartTokenizer.from_pretrained("uclanlp/plbart-python-en_XX", src_lang="python", tgt_lang="en_XX")
plbart_model = PLBartForConditionalGeneration.from_pretrained("uclanlp/plbart-python-en_XX")

# Tokenize the dataset

In [6]:
def fn_tokenize(data):
    new_df = pd.DataFrame(columns=['input_ids_markdown','attention_mask_markdown','input_ids_code','attention_mask_code'])
    for i in data.index:
        new_df.loc[len(new_df)] = [plbarttokenizer(data.loc[i]['markdown'])['input_ids'],plbarttokenizer(data.loc[i]['markdown'])['attention_mask'],plbarttokenizer(data.loc[i]['code'])['input_ids'],plbarttokenizer(data.loc[i]['code'])['attention_mask']]

    return new_df

In [7]:
new_df_1 = fn_tokenize(df_1)
new_df_1

Token indices sequence length is longer than the specified maximum sequence length for this model (2322 > 1024). Running this sequence through the model will result in indexing errors


,input_ids_markdown,attention_mask_markdown,input_ids_code,attention_mask_code
0,"[26343, 5544, 1118, 33475, 18938, 30816, 21498...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[662, 2988, 268, 865, 2, 50002]","[1, 1, 1, 1, 1, 1]"
1,"[272, 5037, 33463, 6423, 375, 34, 261, 168, 33...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[662, 4112, 268, 3880, 2, 50002]","[1, 1, 1, 1, 1, 1]"
2,"[2782, 272, 1139, 395, 2519, 55, 57, 1269, 641...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[320, 2220, 662, 2220, 2, 50002]","[1, 1, 1, 1, 1, 1]"
3,"[126, 632, 11182, 2, 50002]","[1, 1, 1, 1, 1]","[320, 9021, 9, 3945, 662, 24130, 320, 9021, 9,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"[1916, 1487, 144, 661, 71, 57, 759, 2139, 395,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3370, 24, 3880, 33455, 474, 33456, 4424, 2771...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...
7567,"[708, 57, 1677, 2189, 55, 57, 18686, 33463, 14...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[242, 33456, 6160, 24, 2730, 20667, 33455, 542...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
7568,"[4888, 57, 3767, 6306, 4075, 55, 1193, 71, 488...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2133, 1721, 13446, 662, 2118, 33456, 10086, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
7569,"[3051, 6959, 3051, 54, 7510, 27603, 23987, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[662, 4112, 268, 3880, 320, 13446, 33455, 3248...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
7570,"[9475, 6420, 20052, 826, 4591, 2, 50002]","[1, 1, 1, 1, 1, 1, 1]","[6959, 33456, 21042, 24, 10603, 33456, 5034, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [13]:
new_df_1 = new_df_1.sample(frac = 1).reset_index(drop=True)

In [26]:
new_df_1.to_csv("../data/tokenized_data_1.csv")

In [16]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [17]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [71]:
trainer = Trainer(
    model=plbart_model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [63]:
dataset_1 =  load_dataset("csv", data_files={"train": ["../data/train.csv"], "test": "../data/test.csv"})

Using custom data configuration default-ab90834c79f19ed7


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to C:\Users\zdahmani\.cache\huggingface\datasets\csv\default-ab90834c79f19ed7\0.0.0\652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]